In [128]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import time
import datetime
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
from sklearn import utils
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from sklearn import decomposition
import math
from math import sqrt

# Modelling Helpers
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [129]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.dropna(how='any',inplace=True)
test.dropna(how='any',inplace=True)
train = train.reset_index()
test = test.reset_index()
full = train.append( test , ignore_index = True ) 
print("Train data shape:",train.shape,"Test data shape",test.shape)
full.tail()

Train data shape: (355, 12) Test data shape (88, 11)


,categories,genres,id,index,is_free,playtime_forever,price,purchase_date,release_date,tags,total_negative_reviews,total_positive_reviews
438,"Single-player,Steam Achievements,Full controll...",Action,85,85,False,NaN,5000.0,"Mar 23, 2018",22-May-13,"Western,FPS,Action,Story Rich,Bullet Time,Shoo...",516.0,9334.0
439,Single-player,"Action,Adventure",86,86,False,NaN,3600.0,"Mar 3, 2018",14-Sep-09,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",176.0,1245.0
440,"Single-player,Multi-player,Cross-Platform Mult...","Simulation,Strategy",87,87,False,NaN,11200.0,"Feb 18, 2018",13-Aug-13,"Grand Strategy,Strategy,Historical,Simulation,...",6350.0,33128.0
441,"Single-player,Multi-player,Co-op,Online Co-op,...","Action,Adventure,RPG,Simulation,Sports,Strategy",88,88,True,NaN,0.0,"Nov 19, 2017",19-Jun-17,"VR,RPG,Action,Sports,Simulation,Adventure,Stra...",0.0,5.0
442,"Single-player,Multi-player,Steam Achievements,...","Action,Strategy",89,89,False,NaN,26800.0,"Nov 25, 2018",28-Sep-17,"Strategy,Fantasy,Turn-Based Strategy,RTS,Grand...",2594.0,17777.0


In [130]:
# Dummy variables for is_free
Is_free = pd.DataFrame()
Is_free["is_free"] = pd.Series( np.where( full.is_free == True , 1 , 0 ) , name = 'is_free' )

In [131]:
Genres = pd.DataFrame()
Genres['genres'] = full['genres']
Genres = pd.concat([Genres.drop('genres', 1), Genres['genres'].str.get_dummies(sep=",")], 1)
Genres.tail()

,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Utilities,Violent
438,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
439,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
440,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
441,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0
442,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [132]:
Categories = pd.DataFrame()
Categories['categories'] = full['categories']
Categories = pd.concat([Categories.drop('categories', 1), Categories['categories'].str.get_dummies(sep=",")], 1)
Categories.head()

,Captions available,Co-op,Commentary available,Cross-Platform Multiplayer,Full controller support,In-App Purchases,Includes Source SDK,Includes level editor,Local Co-op,Local Multi-Player,...,Single-player,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,1,0,1,1,1,1,0,0,0,0


In [133]:
Tags = pd.DataFrame()
Tags['tags'] = full['tags']
Tags = pd.concat([Tags.drop('tags', 1), Tags['tags'].str.get_dummies(sep=",")], 1)
Tags.head()

,1980s,1990's,2.5D,2D,3D,3D Platformer,3D Vision,4 Player Local,4X,ATV,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
'''
Genres = pd.DataFrame(np.zeros((full.shape[0], 1)))
for x in range(full.shape[0]):
    genres = full['genres'][x].split(',')
    for y in genres:
        y = 'genres_'+y
        if y not in Genres.columns:
            Genres[y] = 0
        Genres[y][x] = 1
#drop the created 0 column
Genres = Genres.drop([0],axis=1)
Genres.head()
'''

"\nGenres = pd.DataFrame(np.zeros((full.shape[0], 1)))\nfor x in range(full.shape[0]):\n    genres = full['genres'][x].split(',')\n    for y in genres:\n        y = 'genres_'+y\n        if y not in Genres.columns:\n            Genres[y] = 0\n        Genres[y][x] = 1\n#drop the created 0 column\nGenres = Genres.drop([0],axis=1)\nGenres.head()\n"

In [135]:
#convert purchase_date into purchase_timestamp and assign to timestamp df
timestamp = pd.DataFrame(np.zeros((full.shape[0], 1)))
Purchase_date = pd.DataFrame(np.zeros((full.shape[0], 1)))
timestamp['purchase_timestamp'] = 0
for x in range(full.shape[0]):
    timestamp['purchase_timestamp'][x] = time.mktime(datetime.datetime.strptime(full['purchase_date'][x], "%b %d, %Y").timetuple())
timestamp["purchase_timestamp"] /= 100000
timestamp["purchase_timestamp"] -= 15700
timestamp["purchase_timestamp"] *= -1
timestamp = preprocessing.scale(timestamp)
Purchase_date['purchase_timestamp'] = pd.DataFrame(timestamp[:,1])
Purchase_date = Purchase_date.drop([0],axis=1)
Purchase_date.head()

,purchase_timestamp
0,-0.373722
1,1.658722
2,-0.373722
3,1.651750
4,0.044620


In [136]:
timestamp2 = pd.DataFrame(np.zeros((full.shape[0], 1)))
Release_date = pd.DataFrame(np.zeros((full.shape[0], 1)))
timestamp2['release_timestamp'] = 0
for x in range(train.shape[0]):
    if x == 290:
        timestamp2['release_timestamp'][x] = time.mktime(datetime.datetime.strptime(full['release_date'][x], "%b %d, %Y").timetuple())
    else:
        timestamp2['release_timestamp'][x] = time.mktime(datetime.datetime.strptime(full['release_date'][x], "%d %b, %Y").timetuple())
for x in range(train.shape[0], full.shape[0]):
    timestamp2['release_timestamp'][x] = time.mktime(datetime.datetime.strptime(full['release_date'][x], "%d-%b-%y").timetuple())
timestamp2["release_timestamp"] /= 100000
timestamp2["release_timestamp"] -= 15700
timestamp2["release_timestamp"] *= -1
timestamp2 = preprocessing.scale(timestamp2)
Release_date['release_timestamp'] = pd.DataFrame(timestamp2[:,1])
Release_date = Release_date.drop([0],axis=1)
Release_date.head()

,release_timestamp
0,0.580497
1,-0.062180
2,0.528872
3,2.002815
4,1.105175


In [137]:
'''
dateTimeNow = datetime.now()
timestampNow = round(datetime.timestamp(dateTimeNow))
timestampNow = str(timestampNow)[0:3]
timestampNow
'''

'\ndateTimeNow = datetime.now()\ntimestampNow = round(datetime.timestamp(dateTimeNow))\ntimestampNow = str(timestampNow)[0:3]\ntimestampNow\n'

In [138]:
#sort the timestamp
#timestamp.sort_values("purchase_timestamp").head(50)

In [139]:
#get current date and current timestamp
'''
from datetime import datetime
dateTimeNow = datetime.now()
timestampNow = round(datetime.timestamp(dateTimeNow))
timestampNow = str(timestampNow)[0:3]
timestampNow
'''

'\nfrom datetime import datetime\ndateTimeNow = datetime.now()\ntimestampNow = round(datetime.timestamp(dateTimeNow))\ntimestampNow = str(timestampNow)[0:3]\ntimestampNow\n'

In [140]:
#scale good reviews
Total_positive_reviews = pd.DataFrame()
Total_positive_reviews["total_positive_reviews"] = np.log(full["total_positive_reviews"]+1)
#total_positive_reviews

In [141]:
#scale bad reviews
Total_negative_reviews = pd.DataFrame()
Total_negative_reviews["total_negative_reviews"] = np.log(full["total_negative_reviews"]+1)
#total_negative_reviews

In [142]:
#price
Price = pd.DataFrame()
Price["price"] = np.log(full["price"]+1)
Price.head()

,price
0,8.216358
1,0.000000
2,8.517393
3,9.200391
4,8.476580


In [143]:
full_X = pd.concat( [ Is_free , Price, Genres, Tags, Categories, Purchase_date, Release_date, Total_positive_reviews , Total_negative_reviews ] , axis=1 )
full_X.head()
pca = decomposition.PCA(n_components=2)
full_X = pca.fit_transform(full_X)
explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.34750057, 0.11666732])

In [144]:
train_X = full_X[0:355]
train_y = full['playtime_forever'][0:355].to_frame()
test_X = full_X[355:]
test_y = full['playtime_forever'][355:].to_frame()
#train_X , valid_X , train_y , valid_y = train_test_split( train_valid_X , train_valid_y , train_size = .8 ) 
#print (train_X_axis.shape , train_X.shape , valid_X.shape , train_y.shape , valid_y.shape)

In [145]:
train_y.head()

,playtime_forever
0,0.000000
1,0.016667
2,0.000000
3,1.533333
4,22.333333


In [146]:
#17.94
# model = Sequential()
# n_cols = train_X.shape[1]

# model.add(Dense(8, activation='relu', input_shape=(n_cols,)))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1))

In [147]:
model = Sequential()
n_cols = train_X.shape[1]

model.add(Dense(30, activation='relu', input_shape=(n_cols,)))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

In [148]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [149]:
early_stopping_monitor = EarlyStopping(patience=3)   
model.fit(train_X, train_y.values, validation_split=0.2, epochs=50, callbacks=[early_stopping_monitor])

temp = pd.DataFrame(np.zeros((train_y.shape[0],1)))
temp["playtime_forever"] = pd.DataFrame(model.predict(train_X))
value = (temp["playtime_forever"] - train_y["playtime_forever"])*(temp["playtime_forever"] - train_y["playtime_forever"])
sqrt(np.sum(value)/train_y.shape[0])

Train on 284 samples, validate on 71 samples
Epoch 1/50
284/284 [==============================] - 0s 334us/step - loss: 145.8029 - val_loss: 74.7993
Epoch 2/50
284/284 [==============================] - 0s 46us/step - loss: 142.4846 - val_loss: 73.0553
Epoch 3/50
284/284 [==============================] - 0s 46us/step - loss: 139.8376 - val_loss: 71.5642
Epoch 4/50
284/284 [==============================] - 0s 46us/step - loss: 136.9348 - val_loss: 70.2686
Epoch 5/50
284/284 [==============================] - 0s 46us/step - loss: 134.2581 - val_loss: 68.9872
Epoch 6/50
284/284 [==============================] - 0s 49us/step - loss: 132.3695 - val_loss: 67.6244
Epoch 7/50
284/284 [==============================] - 0s 46us/step - loss: 129.5186 - val_loss: 66.6465
Epoch 8/50
284/284 [==============================] - 0s 49us/step - loss: 127.9019 - val_loss: 65.6878
Epoch 9/50
284/284 [==============================] - 0s 46us/step - loss: 125.2670 - val_loss: 65.0967
Epoch 10/50
284/28

10.188634299958458

In [150]:
submission = pd.read_csv("samplesubmission.csv")
submission[['playtime_forever']] = submission[['playtime_forever']].astype(float)
print(submission.dtypes)
#result = model.predict(test_X.values)
result = model.predict(test_X)
for idx, a in enumerate(result):
    if a < 0:
        result[idx] = 0
test['playtime_forever'] = result
test.head()

id                    int64
playtime_forever    float64
dtype: object


,index,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,playtime_forever
0,0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...","Oct 28, 2018",22-May-12,2607.0,1122.0,2.234383
1,1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Jul 20, 2019",24-Apr-18,5762.0,2235.0,3.494747
2,2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...","Jul 16, 2019",20-Mar-17,687.0,133.0,0.491894
3,3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical","Jan 30, 2018",17-Jul-07,67.0,39.0,0.661304
4,4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...","Feb 24, 2017",22-Aug-13,40344.0,3708.0,11.688707


In [151]:
for x in test['id']:
    submission['playtime_forever'][x] = test.loc[test['id'] == x]['playtime_forever']
submission.to_csv("FinalResult.csv",index = False)